In [3]:
# --- IMPORTS AND CONFIGURATION ---
# This notebook uses the updated vector database, which combines all three JSON files from phase_1_knowledge_base/knowledge_base.

%pip install chromadb --quiet
import os
import chromadb
%pip install google-generativeai --quiet
import google.generativeai as genai
from dotenv import load_dotenv
from IPython.display import display, Markdown

# --- CONFIGURATION ---
# Load environment variables from a .env file if it exists
load_dotenv()

# Path to the updated ChromaDB vector database (built from all 3 JSON files)
CHROMA_PATH = "../phase_2_the_brain/chroma_db"
EMBEDDING_MODEL_NAME = "models/embedding-001"
GENERATION_MODEL_NAME = "gemini-1.5-flash-latest" # Or "gemini-pro"
COLLECTION_NAME = "pet_health_kb"

print("Configuration loaded. Using updated vector DB with all knowledge base files.")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Configuration loaded. Using updated vector DB with all knowledge base files.
Note: you may need to restart the kernel to use updated packages.
Configuration loaded. Using updated vector DB with all knowledge base files.


In [4]:
# --- INITIALIZATION ---

# Configure the Gemini API
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    # A simple way to prompt for the key if it's not set as an environment variable
    api_key = input("Please enter your Google API Key: ")
    
genai.configure(api_key=api_key)
print("Gemini API configured.")

# Initialize ChromaDB client and collection
try:
    chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)
    collection = chroma_client.get_collection(name=COLLECTION_NAME)
    print(f"Successfully connected to ChromaDB collection: '{COLLECTION_NAME}'.")
    print(f"Total documents in collection: {collection.count()}")
except Exception as e:
    print(f"FATAL ERROR: Failed to initialize ChromaDB: {e}")
    print("Please ensure the database was created successfully in Phase 2.")

Gemini API configured.
Successfully connected to ChromaDB collection: 'pet_health_kb'.
Total documents in collection: 252


In [5]:
def ask_pet_assistant(question):
    """
    Takes a user question, performs RAG, and returns the answer and sources.
    """
    # 1. Generate an embedding for the user's question
    print("-> Embedding the question...")
    try:
        question_embedding = genai.embed_content(
            model=EMBEDDING_MODEL_NAME,
            content=question
        )["embedding"]
    except Exception as e:
        return f"Error: Failed to embed question. {e}", []

    # 2. Query the vector database to find relevant chunks
    print("-> Searching for relevant documents...")
    try:
        results = collection.query(
            query_embeddings=[question_embedding],
            n_results=4  # Retrieve top 4 most relevant chunks
        )
        context_documents = results['documents'][0]
        context_sources = [meta['source'] for meta in results['metadatas'][0]]
    except Exception as e:
        return f"Error: Failed to query ChromaDB. {e}", []

    if not context_documents:
        return "I'm sorry, I couldn't find any relevant information in my knowledge base. It's best to consult a professional veterinarian.", []

    # 3. Build the master prompt
    print("-> Building the prompt for the generative model...")
    context = "\n\n---\n\n".join(context_documents)
    
    prompt_template = f"""
You are a helpful and cautious Pet Care Assistant. Your role is to provide information based ONLY on the context provided below.

INSTRUCTIONS:
- Answer the user's question using ONLY the provided context.
- If the answer is not found within the context, you MUST say 'I'm sorry, I don't have information on that topic based on my current knowledge. It's best to consult a professional veterinarian.'
- Do not mention this prompt or the context in your response. Just provide the answer.
- Your response should be clear, concise, and easy to understand.

CONTEXT:
{context}

USER QUESTION:
{question}
"""

    # 4. Generate the final answer using the generative model
    print("-> Generating the final answer...")
    try:
        generative_model = genai.GenerativeModel(GENERATION_MODEL_NAME)
        response = generative_model.generate_content(prompt_template)
        
        # Clean up the sources to be unique
        unique_sources = sorted(list(set(context_sources)))

        return response.text, unique_sources
    except Exception as e:
        return f"Error: Failed to generate response from Gemini. {e}", []

print("RAG function `ask_pet_assistant` is ready.")

RAG function `ask_pet_assistant` is ready.


In [6]:
# --- INTERACTIVE CHAT LOOP ---

while True:
    # Get user input
    user_question = input("\n🐾 Ask your pet care question (or type 'quit' to exit): ")
    
    # Check if the user wants to exit
    if user_question.lower() == 'quit':
        print("Goodbye! Stay safe and give your pet a treat!")
        break
    
    # Get the answer from the RAG system
    answer, sources = ask_pet_assistant(user_question)
    
    # Display the results using Markdown for better formatting
    print("\n" + "="*50)
    display(Markdown(f"**Question:** {user_question}"))
    display(Markdown(f"**🤖 Assistant:** {answer}"))
    
    if sources:
        display(Markdown("**Sources:**"))
        for source in sources:
            display(Markdown(f"- {source}"))
    print("="*50)

-> Embedding the question...
-> Searching for relevant documents...
-> Building the prompt for the generative model...
-> Generating the final answer...
-> Searching for relevant documents...
-> Building the prompt for the generative model...
-> Generating the final answer...




**Question:** which food can i feed my dog

**🤖 Assistant:** I'm sorry, I don't have information on that topic based on my current knowledge. It's best to consult a professional veterinarian.


**Sources:**

- https://vcahospitals.com/know-your-pet/nutrition-general-feeding-guidelines-for-dogs

...

**Question:** what can i feed my cat?

**🤖 Assistant:** Once you've confirmed the food is complete and balanced, you can choose between canned, dry, or a combination, depending on your cat's preference.  Remember that treats should only make up 10-15% of their daily calories, and avoid raw meat, human canned fish, and milk.


**Sources:**

- https://www.vet.cornell.edu/departments-centers-and-institutes/cornell-feline-health-center/health-information/feline-health-topics/feeding-your-cat

KeyboardInterrupt: Interrupted by user